# Quantitative Value Strategy
"Value investing" means investing in the stocks that are cheapest relative to common measures of business value (like earnings or assets).

For this project, we're going to build an investing strategy that selects the 50 stocks with the best value metrics. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.

## Library Imports
The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np
import pandas as pd
import requests
import math
from scipy import stats
import xlsxwriter

## Importing Our List of Stocks & API Token
As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the .csv file is still in your working directory and import it with the following command:

In [4]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

In [5]:
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


## Making Our First API Call
It's now time to make the first version of our value screener!

We'll start by building a simple value screener that ranks securities based on a single metric (the price-to-earnings ratio).

In [10]:
symbol = "AAPL"
api_url = f"https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}"

data = requests.get(api_url).json()
print(data)

{'symbol': 'AAPL', 'companyName': 'Apple Inc', 'primaryExchange': 'CEGESQ  A)LEKGONT/MLBAT RSANSDL(A', 'calculationPrice': 'close', 'open': 124.59, 'openTime': 1679228166777, 'openSource': 'flocfaii', 'close': 125.99, 'closeTime': 1673108693246, 'closeSource': 'fifclioa', 'high': 128.57, 'highTime': 1650465345710, 'highSource': 'd1ru y5emetpe ni ldiace', 'low': 120.8, 'lowTime': 1689336950420, 'lowSource': 'cu 1dmlpyiaien r tde5ee', 'latestPrice': 124.47, 'latestSource': 'Close', 'latestTime': 'March 18, 2021', 'latestUpdate': 1671047637728, 'latestVolume': 126772169, 'iexRealtimePrice': 123.09, 'iexRealtimeSize': 94, 'iexLastUpdated': 1629994961371, 'delayedPrice': 125.9, 'delayedPriceTime': 1636575841549, 'oddLotDelayedPrice': 124.24, 'oddLotDelayedPriceTime': 1645481005790, 'extendedPrice': 121.54, 'extendedChange': 0.02, 'extendedChangePercent': 0.00018, 'extendedPriceTime': 1664265913509, 'previousClose': 125.28, 'previousVolume': 117077856, 'change': -4.44, 'changePercent': -0.03

## Parsing Our API Call
This API call has the metric we need - the price-to-earnings ratio.

Here is an example of how to parse the metric from our API call:

In [11]:
price = data["latestPrice"]

In [15]:
pe_ratio = data["peRatio"]

In [16]:
pe_ratio

33.01

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called chunks that we can use to divide our list of securities into groups of 100.

In [17]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [150]:
final_dataframe = pd.DataFrame(columns=my_columns)

In [151]:
data = requests.get(batch_api_call_url).json()

In [152]:
for symbol_string in symbol_strings:
    
    batch_api_call_url = f"https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}"
    data = requests.get(batch_api_call_url).json() 
        
    for symbol in symbol_string.split(","):
        final_dataframe = final_dataframe.append(
        pd.Series(
        [symbol,
        data[symbol]['quote']['latestPrice'],
        data[symbol]['quote']['peRatio'],
        "N/A"],
        index=my_columns), ignore_index=True)

In [153]:
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,A,125.95,48.52,N/A
1,AAL,25.20,-1.4,N/A
2,AAP,191.37,26.26,N/A
3,AAPL,120.69,33.95,N/A
4,ABBV,103.89,39.44,N/A
...,...,...,...,...
500,YUM,109.81,37.59,N/A
501,ZBH,161.45,-246.3,N/A
502,ZBRA,471.24,51.44,N/A
503,ZION,58.00,9.27,N/A


## Removing Glamour Stocks

The opposite of a "value stock" is a "glamour stock". 

Since the goal of this strategy is to identify the 50 best value stocks from our universe, our next step is to remove glamour stocks from the DataFrame.

We'll sort the DataFrame by the stocks' price-to-earnings ratio, and drop all stocks outside the top 50.

In [154]:
final_dataframe.sort_values("Price-to-Earnings Ratio", ascending = True, inplace=True)


In [155]:
#Removing stocks with -ve pe ratio
final_dataframe = final_dataframe[final_dataframe["Price-to-Earnings Ratio"] > 0]

final_dataframe = final_dataframe[:50]

final_dataframe.reset_index(inplace=True, drop=True)

In [156]:
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,BIO,590.100,4.7,N/A
1,ALL,117.670,6.7,N/A
2,UNM,28.710,7.39,N/A
3,EBAY,59.700,7.57,N/A
4,AFL,54.350,7.91,N/A
5,COO,379.180,8.41,N/A
6,KIM,19.910,8.5,N/A
7,CE,150.060,8.95,N/A
8,KR,35.240,9.26,N/A
9,ZION,58.000,9.27,N/A


## Calculating the Number of Shares to Buy
We now need to calculate the number of shares we need to buy. 

To do this, we will use the `portfolio_input` function that we created in our momentum project.

I have included this function below.

In [157]:
MAX_TRIES = 10
def portfolio_input():
    tries = 0
    portfolio_size = None
    while portfolio_size is None:
        if tries < MAX_TRIES:
            portfolio_value_string = input("Enter the value of your portfolio... ")
        #     portfolio_value_string = 10000000
            try:
                portfolio_size = float(portfolio_value_string)
                print(portfolio_size)
                return portfolio_size
            
            except ValueError:
                print("That's not a number! Enter a numerical value of your portfolio in $.")
                tries += 1
        else:
            print("Taking 0 as input (-;")
            return 0
    
    return portfolio_size

In [158]:
portfolio_size = portfolio_input()

Enter the value of your portfolio... 1000000
1000000.0


Use the `portfolio_input` function to accept a `portfolio_size` variable from the user of this script.

In [159]:
position_size = float(portfolio_size)/len(final_dataframe.index)

In [160]:
for row in range(0,len(final_dataframe.index)):
    final_dataframe.loc[row, "Number of Shares to Buy"] = math.floor(position_size/final_dataframe.loc[row,"Price"])

You can now use the global `portfolio_size` variable to calculate the number of shares that our strategy should purchase.

## Building a Better (and More Realistic) Value Strategy
Every valuation metric has certain flaws.

For example, the price-to-earnings ratio doesn't work well with stocks with negative earnings.

Similarly, stocks that buyback their own shares are difficult to value using the price-to-book ratio.

Investors typically use a `composite` basket of valuation metrics to build robust quantitative value strategies. In this section, we will filter for stocks with the lowest percentiles on the following metrics:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
* Enterprise Value divided by Gross Profit (EV/GP)

Some of these metrics aren't provided directly by the IEX Cloud API, and must be computed after pulling raw data. We'll start by calculating each data point from scratch.

In [191]:
symbol = "AAPL"

batch_api_call_url = f"https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}"
data = requests.get(batch_api_call_url).json()



In [192]:
data[symbol].keys()

dict_keys(['advanced-stats', 'quote'])

In [193]:
#Price-to-earnings ratio
pe_ratio = data[symbol]['quote']['peRatio']

# Price-to-book ratio
pb_ratio = data[symbol]['advanced-stats']['priceToBook']

# Price-to-sales ratio
ps_ratio = data[symbol]['advanced-stats']['priceToSales']

# Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
ebitda = data[symbol]['advanced-stats']['EBITDA']
gross_profit = data[symbol]['advanced-stats']['grossProfit']

ev_to_ebitda = float(enterprise_value)/float(ebitda)

# Enterprise Value divided by Gross Profit (EV/GP)
ev_to_gross_profit = float(enterprise_value)/float(gross_profit)

Let's move on to building our DataFrame. You'll notice that I use the abbreviation `rv` often. It stands for `robust value`, which is what we'll call this sophisticated strategy moving forward.

In [194]:
rv_columns = [
    "Ticker",
    "Price",
    "Number of Shares to Buy",
    
    "Price-to-Earnings Ratio",
    "PE Percentile",
    
    "Price-to-Book Ratio",
    "PB Ratio Percentile",
    
    "Price-to-Sales Ratio",
    "PS Ratio Percentile",
    
    "EV/EBITDA",    
    "EV/EBITDA Percentile",
    
    "EV/GP",    
    "EV/GP Percentile",
    
    "RV Score"
    
]

In [195]:
rv_dataframe = pd.DataFrame(columns=rv_columns)

In [196]:
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Ratio Percentile,Price-to-Sales Ratio,PS Ratio Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score


In [197]:
ratio_calc = lambda x,y: float(x)/float(y) if (y is not None and x is not None) else np.NaN

for symbol_string in symbol_strings:
    
    batch_api_call_url = f"https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}"
    data = requests.get(batch_api_call_url).json()
    
    for symbol in symbol_string.split(","):
        #Price-to-earnings ratio
        pe_ratio = data[symbol]['quote']['peRatio']

        # Price-to-book ratio
        pb_ratio = data[symbol]['advanced-stats']['priceToBook']

        # Price-to-sales ratio
        ps_ratio = data[symbol]['advanced-stats']['priceToSales']

        # Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
        enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
        
        ebitda = data[symbol]['advanced-stats']['EBITDA']
        gross_profit = data[symbol]['advanced-stats']['grossProfit']

        ev_to_ebitda = ratio_calc(enterprise_value, ebitda)
        

        # Enterprise Value divided by Gross Profit (EV/GP)
        ev_to_gross_profit = ratio_calc(enterprise_value, gross_profit)
        
        rv_dataframe = rv_dataframe.append(pd.Series([
            
                                        symbol,
                                        data[symbol]['quote']['latestPrice'],
                                        "N/A",

                                        pe_ratio,
                                        "N/A",

                                        pb_ratio,
                                        "N/A",

                                        ps_ratio,
                                        "PS Ratio Percentile",

                                        ev_to_ebitda,    
                                        "N/A",

                                        ev_to_gross_profit,    
                                        "N/A",

                                        "N/A"
                                        
                                      ], 
                                      index=rv_columns), 
                            ignore_index = True)
    


## Dealing With Missing Data in Our DataFrame

Our DataFrame contains some missing data because all of the metrics we require are not available through the API we're using. 

You can use pandas' `isnull` method to identify missing data:

In [198]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Ratio Percentile,Price-to-Sales Ratio,PS Ratio Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
40,AON,225.13,N/A,None,N/A,None,N/A,None,PS Ratio Percentile,NaN,N/A,NaN,N/A,N/A
42,APA,19.73,N/A,None,N/A,None,N/A,None,PS Ratio Percentile,NaN,N/A,NaN,N/A,N/A
71,BRK.B,256.47,N/A,None,N/A,None,N/A,None,PS Ratio Percentile,NaN,N/A,NaN,N/A,N/A
118,CTL,11.00,N/A,9.97,N/A,None,N/A,None,PS Ratio Percentile,NaN,N/A,NaN,N/A,N/A
136,DISCK,65.25,N/A,None,N/A,None,N/A,None,PS Ratio Percentile,NaN,N/A,NaN,N/A,N/A
165,ETFC,51.14,N/A,14.28,N/A,None,N/A,None,PS Ratio Percentile,NaN,N/A,NaN,N/A,N/A
168,EVRG,58.70,N/A,None,N/A,None,N/A,None,PS Ratio Percentile,NaN,N/A,NaN,N/A,N/A
190,FOX,40.90,N/A,None,N/A,None,N/A,None,PS Ratio Percentile,NaN,N/A,NaN,N/A,N/A
192,FRC,177.74,N/A,None,N/A,None,N/A,None,PS Ratio Percentile,NaN,N/A,NaN,N/A,N/A
204,GOOG,2053.39,N/A,None,N/A,None,N/A,None,PS Ratio Percentile,NaN,N/A,NaN,N/A,N/A


Dealing with missing data is an important topic in data science.

There are two main approaches:

* Drop missing data from the data set (pandas' `dropna` method is useful here)
* Replace missing data with a new value (pandas' `fillna` method is useful here)

In this tutorial, we will replace missing data with the average non-`NaN` data point from that column. 

Here is the code to do this:

In [199]:
numerical_columns_to_impute = [     
                                "Price-to-Earnings Ratio",

                                "Price-to-Book Ratio",

                                "Price-to-Sales Ratio",

                                "EV/EBITDA",    

                                "EV/GP"]

for column in numerical_columns_to_impute:
    rv_dataframe[column].fillna(rv_dataframe[column].mean(), inplace=True)

Now, if we run the statement from earlier to print rows that contain missing data, nothing should be returned:

In [200]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Ratio Percentile,Price-to-Sales Ratio,PS Ratio Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score


## Calculating Value Percentiles

We now need to calculate value score percentiles for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* EV/EBITDA
* EV/GP

Here's how we'll do this:

In [201]:
from scipy.stats import percentileofscore as score

metrics_mapping = {    
    "Price-to-Earnings Ratio": "PE Percentile",
    
    "Price-to-Book Ratio": "PB Ratio Percentile",
    
    "Price-to-Sales Ratio": "PS Ratio Percentile",
    
    "EV/EBITDA": "EV/EBITDA Percentile",
    
    "EV/GP": "EV/GP Percentile"
}


for metric in metrics_mapping.keys():
    for row in rv_dataframe.index:
        rv_dataframe.loc[row, metrics_mapping[metric]] = score(rv_dataframe[metric], 
                                                               rv_dataframe.loc[row, metric])/100

In [202]:
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Ratio Percentile,Price-to-Sales Ratio,PS Ratio Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,127.94,N/A,47.21,0.825743,7.83,0.734653,6.8900,0.784158,30.658079,0.867327,13.513012,0.79604,N/A
1,AAL,25.30,N/A,-1.37,0.154455,-2.30,0.0455446,0.9223,0.0772277,-4.559917,0.039604,2.404328,0.0910891,N/A
2,AAP,184.10,N/A,26.17,0.485149,3.55,0.445545,1.2500,0.141584,11.861500,0.263366,2.614204,0.10495,N/A
3,AAPL,125.22,N/A,34.37,0.679208,30.85,0.952475,6.9500,0.789109,23.271316,0.742574,17.348257,0.90099,N/A
4,ABBV,108.62,N/A,39.91,0.772277,14.51,0.879208,4.0000,0.555446,13.941936,0.364356,8.334779,0.49505,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,111.23,N/A,38.13,0.746535,-4.32,0.0435644,5.7800,0.728713,23.659627,0.756436,9.951210,0.643564,N/A
501,ZBH,161.06,N/A,-239.90,0.00990099,2.84,0.363366,4.8700,0.657426,30.555217,0.861386,8.221355,0.489109,N/A
502,ZBRA,468.75,N/A,50.67,0.841584,11.80,0.843564,5.7800,0.728713,32.957299,0.885149,12.520564,0.744554,N/A
503,ZION,59.00,N/A,9.04,0.176238,1.30,0.109901,2.7200,0.391089,6.952278,0.0673267,3.032666,0.124752,N/A


In [203]:
metrics

{'Price-to-Earnings Ratio': 'PE Percentile',
 'Price-to-Book Ratio': 'PB Ratio Percentile',
 'Price-to-Sales Ratio': 'PS Ratio Percentile',
 'EV/EBITDA': 'EV/EBITDA Percentile',
 'EV/GP': 'EV/GP Percentile'}

## Calculating the RV Score
We'll now calculate our RV Score (which stands for Robust Value), which is the value score that we'll use to filter for stocks in this investing strategy.

The RV Score will be the arithmetic mean of the 4 percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the mean function from Python's built-in statistics module.

In [204]:
from statistics import mean

for row in rv_dataframe.index:
    metric_percentiles = []
    for metric in metrics_mapping.keys():
        metric_percentiles.append(rv_dataframe.loc[row, metrics[metric]])
    rv_dataframe.loc[row, "RV Score"] = mean(metric_percentiles)

## Selecting the 50 Best Value Stocks¶

As before, we can identify the 50 best value stocks in our universe by sorting the DataFrame on the RV Score column and dropping all but the top 50 entries.

In [205]:
rv_dataframe.sort_values("RV Score", ascending = True, inplace=True)

In [206]:
rv_dataframe = rv_dataframe[:50]

In [207]:
rv_dataframe.reset_index(drop = True, inplace = True)

In [208]:
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Ratio Percentile,Price-to-Sales Ratio,PS Ratio Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,UNM,29.47,N/A,7.46,0.166337,0.5598,0.0475248,0.4534,0.0217822,3.679076,0.0475248,0.446367,0.0039604,0.0574257
1,MCK,195.07,N/A,-7.72,0.110891,-62.7300,0.0039604,0.1276,0.00594059,7.273811,0.0732673,2.811850,0.112871,0.0613861
2,FTI,7.90,N/A,-1.12,0.158416,0.8798,0.0673267,0.2815,0.00990099,2.285674,0.0435644,1.465507,0.0435644,0.0645545
3,AIG,49.36,N/A,-7.01,0.114851,0.6346,0.0534653,0.9905,0.0871287,7.699308,0.0831683,0.949589,0.0217822,0.0720792
4,MET,62.11,N/A,11.31,0.19604,0.7416,0.0554455,0.8369,0.0693069,5.441800,0.0514851,0.814779,0.0158416,0.0776238
5,ALL,119.06,N/A,7.00,0.164356,1.2500,0.10198,0.8188,0.0653465,2.563014,0.0455446,0.761447,0.0138614,0.0782178
6,AAL,25.30,N/A,-1.37,0.154455,-2.3000,0.0455446,0.9223,0.0772277,-4.559917,0.039604,2.404328,0.0910891,0.0815842
7,DXC,28.52,N/A,-2.60,0.140594,1.3000,0.109901,0.4084,0.019802,5.452527,0.0534653,2.368537,0.0851485,0.0817822
8,F,12.71,N/A,-40.10,0.0514851,1.6600,0.187129,0.3879,0.0178218,6.343918,0.0594059,2.866139,0.116832,0.0865347
9,ABC,117.10,N/A,-7.39,0.112871,-35.3000,0.0118812,0.1261,0.0039604,8.602526,0.108911,4.217255,0.209901,0.089505


## Calculating the Number of Shares to Buy
We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a for loop to calculate the number of shares to buy for each stock in our investment universe.

In [209]:
portfolio_size = portfolio_input()

Enter the value of your portfolio... 
That's not a number! Enter a numerical value of your portfolio in $.
Enter the value of your portfolio... 
That's not a number! Enter a numerical value of your portfolio in $.
Enter the value of your portfolio... 2500000
2500000.0


In [210]:
position_size = float(portfolio_size)/len(rv_dataframe.index)

In [211]:
position_size

50000.0

In [212]:
for i in rv_dataframe.index:
    rv_dataframe.loc[i , "Number of Shares to Buy"] = math.floor(position_size/rv_dataframe.loc[i, "Price"])

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [213]:
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Ratio Percentile,Price-to-Sales Ratio,PS Ratio Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,UNM,29.47,1696,7.46,0.166337,0.5598,0.0475248,0.4534,0.0217822,3.679076,0.0475248,0.446367,0.0039604,0.0574257
1,MCK,195.07,256,-7.72,0.110891,-62.7300,0.0039604,0.1276,0.00594059,7.273811,0.0732673,2.811850,0.112871,0.0613861
2,FTI,7.90,6329,-1.12,0.158416,0.8798,0.0673267,0.2815,0.00990099,2.285674,0.0435644,1.465507,0.0435644,0.0645545
3,AIG,49.36,1012,-7.01,0.114851,0.6346,0.0534653,0.9905,0.0871287,7.699308,0.0831683,0.949589,0.0217822,0.0720792
4,MET,62.11,805,11.31,0.19604,0.7416,0.0554455,0.8369,0.0693069,5.441800,0.0514851,0.814779,0.0158416,0.0776238
5,ALL,119.06,419,7.00,0.164356,1.2500,0.10198,0.8188,0.0653465,2.563014,0.0455446,0.761447,0.0138614,0.0782178
6,AAL,25.30,1976,-1.37,0.154455,-2.3000,0.0455446,0.9223,0.0772277,-4.559917,0.039604,2.404328,0.0910891,0.0815842
7,DXC,28.52,1753,-2.60,0.140594,1.3000,0.109901,0.4084,0.019802,5.452527,0.0534653,2.368537,0.0851485,0.0817822
8,F,12.71,3933,-40.10,0.0514851,1.6600,0.187129,0.3879,0.0178218,6.343918,0.0594059,2.866139,0.116832,0.0865347
9,ABC,117.10,426,-7.39,0.112871,-35.3000,0.0118812,0.1261,0.0039604,8.602526,0.108911,4.217255,0.209901,0.089505


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [214]:
writer = pd.ExcelWriter("robust_value_strategy.xlsx", engine = "xlsxwriter")
rv_dataframe.to_excel(writer, sheet_name="Robust Value Strategy", index=False)

## Creating the Formats We'll Need For Our .xlsx File
You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase
* Float formats with 1 decimal for each valuation metric

Since we already built some formats in past sections of this course, I've included them below for you. Run this code cell before proceeding.

In [215]:
background_color = '#ffffff'
font_color = '#0a0a23'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )


float_template = writer.book.add_format(
        {
            'num_format':'0.0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )



percent_template = writer.book.add_format(
        {
            'num_format':'00.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [216]:
column_formats = {
    "A": ["Ticker", string_template],
    "B": ["Price", dollar_template],
    "C": ["Number of Shares to Buy", integer_template],
    "D": ["Price-to-Earnings Ratio", float_template],
    "E": ["PE Percentile", percent_template],
    "F": ["Price-to-Book Ratio", float_template],
    "G": ["PB Ratio Percentile", percent_template],
    "H": ["Price-to-Sales Ratio", float_template],
    "I": ["PS Ratio Percentile", percent_template],
    "J": ["EV/EBITDA", float_template],    
    "K": ["EV/EBITDA Percentile", percent_template],
    "L": ["EV/EBITDA", float_template],    
    "M": ["EV/EBITDA Percentile", percent_template],
    "N": ["RV Score", percent_template]
    
}

for column in column_formats.keys():
    writer.sheets["Robust Value Strategy"].set_column(f"{column}:{column}", 25 , column_formats[column][1])
    writer.sheets["Robust Value Strategy"].write(f"{column}1", column_formats[column][0], column_formats[column][1])


## Saving Our Excel Output
As before, saving our Excel output is very easy:

In [218]:
writer.save()